In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
#from implementations import *
#from split_data import * 
%load_ext autoreload
%autoreload 2


In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

In [3]:
# Main external library : Natural Language Toolkit (nltk)
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()

nltk.download('punkt')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/teframartin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load positive tweets & remove duplicate

In [4]:
train_pos_class = []

In [5]:
filename1 = 'data/train_pos.txt'
with open(filename1) as f1:
    train_pos = f1.readlines()
len(train_pos)

100000

In [6]:
#Remove duplciate positive tweets
train_pos = list(dict.fromkeys(train_pos))
len(train_pos)

90233

In [7]:
#Put each positive tweet in lower case in its own list with correct label (1)
for line in train_pos:
    train_pos_class.append([line.lower(),1])

In [15]:
train_pos_class.shape

AttributeError: 'list' object has no attribute 'shape'

# Load negative tweets & remove duplicate

In [9]:
train_neg_class = []

In [10]:
filename2 = 'data/train_neg.txt'
with open(filename2) as f2:
    train_neg = f2.readlines()
len(train_neg)

100000

In [11]:
#Remove duplciate negative tweets
train_neg = list(dict.fromkeys(train_neg))
len(train_neg)

91088

In [12]:
#Put each negative tweet in lower case in its own list with correct label (0)
for line in train_neg:
    train_neg_class.append([line.lower(),1])

# Cleaning Data

In [13]:
from collections import Counter
import re
import string
nltk.download('wordnet')
nltk.download('omw-1.4')
pd.set_option('display.max_colwidth',100)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/teframartin/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/teframartin/nltk_data...


In [14]:
# A word that is so common that there is no need to use it in a search
ENGLISH_STOP_WORDS = nltk.corpus.stopwords.words('english')

# Adding few extra stop word
ENGLISH_STOP_WORDS = ENGLISH_STOP_WORDS + ['im', 'dont','dunno', 'cant', ' 2 ', "'s", ' u ', ' x ']

print(ENGLISH_STOP_WORDS)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/Users/teframartin/nltk_data'
    - '/Users/teframartin/anaconda3/nltk_data'
    - '/Users/teframartin/anaconda3/share/nltk_data'
    - '/Users/teframartin/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
#Concatenate the two training sets of positive and negative tweets
for line in train_neg_class:
    train_pos_class.append(line)

In [ ]:
train_data = train_pos_class

In [ ]:
train_df = pd.DataFrame(train_data)

#Labeling
train_df.columns = ["tweet", "labels"]

In [ ]:
train_df.head()

In [ ]:
# Calculate the most common words used in the set of all tweets
def get_most_common_words(txt,limit):
    return Counter(txt.split()).most_common()[:limit]

In [ ]:
whole_tweets = " ".join(train_df.tweet.values)
get_most_common_words(whole_tweets,10)

In [ ]:
# Remove from tweets the punctuation and stop words (= a word that is so common that there is no need to use it in a search.)
def clean_tweet(tweet):
    tweet = "".join([w for w in tweet if w not in string.punctuation])
    tokens = re.split('\W+', tweet)
    tweet = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return tweet

In [ ]:
# Apply the clean_tweet transformation
train_df['tweet'] = train_df['tweet'].apply(lambda x : clean_tweet(x))
train_df.head()

In [ ]:
# Change any word belonging to the same word-family into a common word (changing/changes/changed.. ==> change)
def lemmatization(token_tweet):
    tweet = [wn.lemmatize(word) for word in token_tweet]
    return tweet

In [ ]:
# Apply the lemmatization transformation
train_df['tweet'] = train_df['tweet'].apply(lambda x : lemmatization(x))
train_df.head()

In [ ]:
# Concatenate the tokennized tweet into a all text like at the beginning
def concatenate(lst):
    concatenate_tweet = ''
    for elem in lst:
        concatenate_tweet = concatenate_tweet + ' ' + elem
    return concatenate_tweet

In [ ]:
train_df['tweet'] = train_df['tweet'].apply(lambda x : concatenate(x))
train_df.head()

In [ ]:
whole_tweets1 = " ".join(train_df.tweet.values)
get_most_common_words(whole_tweets1,10)